<table align="left" style="border-style: hidden" class="table"> <tr> <td class="col-md-2"><img style="float" src="http://prob140.org/assets/icon256.png" alt="Prob140 Logo" style="width: 120px;"/></td><td><div align="left"><h3 style="margin-top: 0;">Probability for Data Science</h3><h4 style="margin-top: 20px;">UC Berkeley, Spring 2018</h4><p>Ani Adhikari</div></td></tr></table><!-- not in pdf -->

In [ ]:
from datascience import *
from prob140 import *
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
from scipy import stats
from sympy import *
init_printing()
from matplotlib import patches
from mpl_toolkits.mplot3d.axes3d import Axes3D
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
def plot_ellipse(ax, _a, _b, color, draw_lines=True):
    ax.add_artist(patches.Ellipse((0, 0), 6 * _a, 6 * _b, color=color, lw=2, fill=False))
    for x in range(-3, 4):
        ax.plot([-3 * _a, 3 * _a], [x * _b, x * _b], color='k', lw=1)
        ax.plot([x * _a, x * _a], [-3 * _b, 3 * _b], color='k', lw=1)

In [ ]:
def z_to_v(_a, _b, n=0, **kwargs):
    m = max(_a, _b, 1) * 3 + 0.5
    samples = stats.multivariate_normal.rvs(mean=[0, 0], size=n).reshape((n, 2))
    
    def indep_normals(v, w):
        return 1/(2*np.pi*np.abs(_a * _b)) * np.exp(-0.5*((v/_a)**2 + (w/_b)**2))
    def indep_standard_normals(x,y):
        return 1/(2*np.pi) * np.exp(-0.5*(x**2 + y**2))

    x = np.linspace(-m, m, 100)
    y = np.linspace(-m, m, 100)
    X, Y = np.meshgrid(x, y)

    f1 = np.vectorize(indep_standard_normals)
    f2 = np.vectorize(indep_normals)

    Z1 = f1(X, Y)
    Z2 = f2(X, Y)
    
    def plot(azim):
        fig = plt.figure(figsize=(18, 18))
        ax1 = fig.add_subplot(221, projection='3d')
        ax2 = fig.add_subplot(222, projection='3d')
        ax1.plot_surface(X, Y, Z1, color='blue', **kwargs)
        ax2.plot_surface(X, Y, Z2, color='gold', **kwargs)
        z_max = max(Z1.max(), Z2.max())
        ax1.set_zlim3d(0, z_max) 
        ax2.set_zlim3d(0, z_max)
        ax1.set_title('$\mathbf{Z}$')
        ax1.set_xlabel('$z_1$')
        ax1.set_ylabel('$z_2$')
        ax1.set_zlabel('$f(z_1, z_2)$')
        ax2.set_title('$\mathbf{V}$')
        ax2.set_xlabel('$v_1$')
        ax2.set_ylabel('$v_2$')
        ax2.set_zlabel('$f_V(v_1, v_2)$')
        ax1.view_init(20, azim)
        ax2.view_init(20, azim)
        ax3 = fig.add_subplot(223)
        ax4 = fig.add_subplot(224)
        ax3.set_aspect('equal', 'datalim')
        ax4.set_aspect('equal', 'datalim')
        plot_ellipse(ax3, 1, 1, 'blue')
        ax3.scatter(samples[:, 0], samples[:, 1], s=30)
        plot_ellipse(ax4, _a, _b, 'gold')
        ax4.scatter(samples[:, 0] * _a, samples[:, 1] * _b, s=30)
        ax3.set_xlim(-m, m)
        ax3.set_ylim(-m, m)
        ax4.set_xlim(-m, m)
        ax4.set_ylim(-m, m)
    azimuth_slider = widgets.IntSlider(
        value=-90,
        min=-180,
        max=180,
        step=15,
        description='azimuth',
        continuous_update=False
    )

    @interact(azim=azimuth_slider)
    def wrapper(azim):
        plot(azim)

In [ ]:
def unit_square_to_rectangle():
    aa = 2
    bb = 3
    plt.axes().set_aspect('equal')
    plt.xlim(-0.1, aa+0.1)
    plt.xticks(np.arange(0, aa+0.1,))
    plt.ylim(-0.1, bb+0.1)
    plt.yticks(np.arange(0, bb+0.1))
    plt.plot([0, 1], [0, 0], color='k', lw=2)
    plt.plot([0, 0], [0, 1], color='k', lw=2)
    plt.plot([0, 1], [1, 1], color='k', lw=2)
    plt.plot([1, 1], [0, 1], color='k', lw=2)
    plt.plot([0, aa], [0, 0], color='gold', lw=2)
    plt.plot([0, 0], [0, bb], color='gold', lw=2)
    plt.plot([0, aa], [bb, bb], color='gold', lw=2)
    plt.plot([aa, aa], [0, bb], color='gold', lw=2);

In [ ]:
def unit_square_to_parallelogram():
    aa = 3
    bb = 1.5
    plt.axes().set_aspect('equal')
    plt.xlim(-0.1, aa+0.1)
    plt.xticks(np.arange(0, aa+0.1))
    plt.ylim(-0.1, aa+bb+0.1)
    plt.plot([0, 1], [0, 0], color='k', lw=2)
    plt.plot([0, 0], [0, 1], color='k', lw=2)
    plt.plot([0, 1], [1, 1], color='k', lw=2)
    plt.plot([1, 1], [0, 1], color='k', lw=2)
    plt.plot([0, 1], [0, aa], color='red', lw=2)
    plt.plot([0, 0], [0, bb], color='red', lw=2)
    plt.plot([1, 1], [aa, aa+bb], color='red', lw=2)
    plt.plot([0, 1], [bb, aa+bb], color='red', lw=2);

In [ ]:
def z_to_w(_a, _b, n=0, **kwargs):
    m = max(3 * (_a + _b), 4) + 0.5
    samples = stats.multivariate_normal.rvs(mean=[0, 0], size=n).reshape((n, 2))
    cov1 = np.identity(2)
    norm1 = stats.multivariate_normal(cov=cov1)
    cov2 = np.array([[1, _a],[_a, _a**2 + _b**2]])
    norm2 = stats.multivariate_normal(cov=cov2)
    x = np.linspace(-m, m, 100)
    y = np.linspace(-m, m, 100)
    X, Y = np.meshgrid(x, y)
    pos = np.empty(X.shape + (2,))
    pos[:, :, 0] = X
    pos[:, :, 1] = Y
    Z1 = norm1.pdf(pos)
    Z2 = norm2.pdf(pos)
    def plot(azim):
        fig = plt.figure(figsize=(18, 18))
        ax1 = fig.add_subplot(221, projection='3d')
        ax2 = fig.add_subplot(222, projection='3d')
        ax1.plot_surface(X, Y, Z1, color='blue', **kwargs)
        ax2.plot_surface(X, Y, Z2, color='r', **kwargs)
        z_max = max(Z1.max(), Z2.max())
        ax1.set_zlim3d(0, z_max) 
        ax2.set_zlim3d(0, z_max)
        ax1.set_title('$\mathbf{Z}$')
        ax1.set_xlabel('$z_1$')
        ax1.set_ylabel('$z_2$')
        ax1.set_zlabel('$f(z_1, z_2)$')
        ax2.set_title('$\mathbf{W}$')
        ax2.set_xlabel('$w_1$')
        ax2.set_ylabel('$w_2$')
        ax2.set_zlabel('$f_W(w_1, w_2)$')
        ax1.set_xlim(-m, m)
        ax1.set_ylim(-m, m)
        ax2.set_xlim(-m, m)
        ax2.set_ylim(-m, m)
        ax1.view_init(20, azim)
        ax2.view_init(20, azim)
        ax1 = fig.add_subplot(223)
        ax1.set_aspect('equal', 'datalim')
        ax1.set_xlabel('$Z_1$')
        ax1.set_ylabel('$Z_2$')
        ax1.scatter(samples[:, 0], samples[:, 1], s=30)
        plot_ellipse(ax1, 1, 1, 'blue')
        ax2 = fig.add_subplot(224)
        ax2.set_aspect('equal', 'datalim')
        for x in range(-3, 4):
            ax2.plot([-3, 3], [-3 * _a + _b * x, 3 * _a + _b * x], color='k', lw=1)
            ax2.plot([x, x], [_a * x - 3 * _b, _a * x + 3 * _b], color='k', lw=1)
        radius = 32 / (_a ** 2 + _b ** 2) ** 0.5
        ax2.scatter(samples[:, 0], _a * samples[:, 0] + _b * samples[:, 1], s=30)
        ax2.set_xlabel('$W_1$')
        ax2.set_ylabel('$W_2$')
        ax1.set_xlim(-m, m)
        ax1.set_ylim(-m, m)
        ax2.set_xlim(-m, m)
        ax2.set_ylim(-m, m)
        theta = np.linspace(0, 2 * np.pi)
        x = np.cos(theta)
        y = np.sin(theta)
        plt.plot(3 * x, 3 * (_a * x + _b * y), lw=2, color='r')
    
    azimuth_slider = widgets.IntSlider(
        value=-90,
        min=-180,
        max=180,
        step=15,
        description='azimuth',
        continuous_update=False
    )

    @interact(azim=azimuth_slider)
    def wrapper(azim):
        plot(azim)
        


# Lab 10: Introduction to Jointly Normal Vectors #
The multivariate normal distribution is central to many topics in statistical learning theory. In this lab you will develop the most commonly used formula for the multivariate normal joint density.

The lab is designed as a sequence of small steps that lead you to the joint distribution of linear combinations of independent normal variables. That's the multivariate normal distribution. Random variables with this joint distribution are called *jointly normal*. In class we will study the fundamental properties of this joint distribution and its use in multiple linear regression.

The lab is just an introduction to the multivariate normal. It's not intended as a thorough account, and you might have questions at the end of it. Keep a note of your questions. I hope that they will be answered once we study the distribution in class.

**Please make sure you have some paper and a pencil as you do this lab.** It combines algebra, geometry, calculus, linear algebra, and of course probability theory. Working out some steps on scratch paper will be helpful.

What you will learn:
- How to use `SymPy` to do symbolic math including linear algebra
- How to find the joint density of a linear transformation of two variables
- How to parametrize the joint density of two independent normal variables so that the formula can easily be extended to handle dependence of a particular kind as well as higher dimensions

Let's get started with a couple of preliminaries. The first one sets up some standard notation for collections of random variables and their moments. The second one examines the form of the normal density function in one dimension.

## Two Dimensions ##
We are used to thinking about a pair of random variables $X$ and $Y$ as a point $(X, Y)$ on the plane. As we move to higher dimensions, it becomes convenient to use matrix representations instead. 

This lab is almost entirely about two dimensions, so the definitions below are about a pair of random variables. All the definitions have obvious extensions to $n$ dimensions for $n > 2$.

### Random Vector ###
We will represent random variables $X_1$ and $X_2$ as a column vector.
$$
\mathbf{X} ~ = ~ 
\begin{bmatrix}
X_1 \\
X_2 \\
\end{bmatrix}
$$

### Mean Vector ###
The *mean vector* of $\mathbf{X}$ is defined as
$$
\boldsymbol{\mu} ~ = ~ 
\begin{bmatrix}
E(X_1) \\
E(X_2) \\
\end{bmatrix}
$$

### Covariance Matrix ###
The *covariance matrix* of $\mathbf{X}$ is the $2\times2$ matrix $\boldsymbol{\Sigma}$ whose $(i, j)$th element is $Cov(X_i, X_j)$. By properties of covariance,

$$
\boldsymbol{\Sigma} ~ = ~ 
\begin{bmatrix}
Var(X_1) & Cov(X_1, X_2) \\
Cov(X_1, X_2) & Var(X_2)
\end{bmatrix}
$$

Every covariance matrix is symmetric, and the elements along its main diagonal are non-negative. Covariance matrices have another property that isn't easy to spot just by looking. As you will soon see in class, there is a relation between $Cov(X_1, X_2)$, $Var(X_1)$, and $Var(X_2)$; the  relation implies that only positive semi-definite matrices can be covariance matrices. But you don't have to worry about that in this lab.

## The Normal Density Function ##
The normal density function can be written in many ways. Let's look at how we have written it thus far. The goal is to help you focus on the most important aspects of the density function without getting caught up in details.

### Quadratic in the Exponent  ###
The normal $(\mu, \sigma^2)$ density has a quadratic expression in the exponent.

$$
f(x) ~ = ~ \frac{1}{\sqrt{2\pi}} e^{-\frac{1}{2} \big{(} \frac{x-\mu}{\sigma} \big{)}^2} ~ = ~ C e^{-\big{[}\frac{1}{2}(ax^2 + bx + c)\big{]}}, ~~~~~~ -\infty < x < \infty
$$

where $C$, $a$, $b$, and $c$ are constants with $a > 0$. 

The quadratic expression $\frac{1}{2}(ax^2 + bx + c)$ determines the shape of the density. As you know from class, $C$ just makes the density integrate to 1.

If you have a density function and are trying to decide whether it is a normal density, ignore the constants and just look in the exponent. If what you see there is a quadratic, the density is normal. You can work out the mean and SD by methods such as "completing the square", but that exercise in algebra is not part of this lab.

### Centering ###
The focus of the lab is on the form of the normal density function and the shape of the density surface of two jointly normal variables. Shifting the variables only affects location, not the shape. So we will start with centered variables, that is, variables with expectation 0. At the end of the lab we will make a straightforward modification for the case of non-zero means. For now, just assume $\mu = 0$, so that

$$
f(x) ~ = ~ Ce^{-\frac{1}{2}ax^2}, ~~~~~~ -\infty < x < \infty
$$

#newpage

## Part 1. Independent Standard Normal Variables ##

Let $Z_1, Z_2$ be i.i.d. standard normal. As you know, their joint density is

$$
f(z_1, z_2) ~ = ~ \frac{1}{2\pi} e^{-\frac{1}{2}(z_1^2 + z_2^2)}, ~~~~~ -\infty < z_1, z_2 < \infty
$$

Notice that the expression in the exponent is quadratic in $z_1$ and $z_2$. It's a particularly simple quadratic – there are no $z_1z_2$, $z_1$, or $z_2$ terms. That is because $Z_1$ and $Z_2$ are independent and in standard units.

In this part of the lab you will learn how to use `SymPy` and how to rewrite $f$ using matrices.

### 1a) The Familiar Form ###
Run the cell below. It contains the code used in the textbook to plot $f$. Notice the use of `np.pi` for $\pi$ and `np.exp` for exponentiation.

In [ ]:
def indep_standard_normals(x, y):
    return 1/(2*np.pi) * np.exp(-0.5*(x**2 + y**2))

Plot_3d((-4, 4), (-4, 4), indep_standard_normals, rstride=4, cstride=4)
plt.title('Joint Density of $Z_1$ and $Z_2$');

### 1b) Integrals in `SymPy` ###

Let's get started with some symbolic math using `SymPy`. To tell `SymPy` that a variable should be treated as algebraic, you have to `declare` it.

In [ ]:
declare('z_1')
declare('z_2')

The cell below defines $f$ in `SymPy`. Notice the use of `SymPy`'s `pi` and `exp` instead of the corresponding `NumPy` functions. We will alert you about which one you should use.

In [ ]:
f = 1/(2*pi) * exp(-0.5*(z_1**2 + z_2**2))
f

`SymPy` doesn't display the function in the way that's most natural for probability theory. This happens sometimes when we use symbolic math programs. 

Let `function_name` be a `SymPy` function of two variables `x` and `y`. To define the integral of the function over a region in the plane, use:

`Integral(function_name, (x, lower_x, upper_x), (y, lower_y, upper_y))`

where `lower` and `upper` are the lower and upper limits of integration. You can switch the order of integration as needed.

`SymPy` rather cutely uses the symbol `oo` (two lower-case letter o's) for $\infty$ and `-oo` for $-\infty$.

Define the integral of the function $f$ over the entire plane. 

In [ ]:

total_integral = ...
total_integral

To compute the value of a definite integral, use the method `.doit()`. Run the cell below to see that `SymPy` gets the right answer.

In [ ]:
total_integral.doit()

### 1c) Matrices in `SymPy` ###
Let
$$
\mathbf{Z} ~ = ~
\begin{bmatrix}
Z_1 \\
Z_2
\end{bmatrix}
$$
and let
$$
\mathbf{z} ~ = ~
\begin{bmatrix}
z_1 \\
z_2
\end{bmatrix}
$$
be a generic value of $\mathbf{Z}$.

Recall the formula for the joint density of $\mathbf{Z}$:

$$
f(z_1, z_2) ~ = ~ \frac{1}{2\pi} e^{-\frac{1}{2}(z_1^2 + z_2^2)}
$$

We will use the notation $\mathbf{M}^T$ for the transpose of a matrix $\mathbf{M}$. It is clear by algebra that

$$
f(\mathbf{z}) 
~ = ~ \frac{1}{(\sqrt{2\pi})^2} \exp(-\frac{1}{2}\mathbf{z}^T \mathbf{z})
$$

Let's see how to calculate $\mathbf{z}^T\mathbf{z}$ in `SymPy`.

The function `Matrix` converts a list into a column matrix. The method `.T` returns the transpose of the matrix.

In [ ]:
z = Matrix([z_1, z_2])
z

In [ ]:
z.T

Compute $\mathbf{z}^T \mathbf{z}$ in the cell below. The ordinary multiplication symbol `*` works for matrix multiplication in `SymPy`.

In [ ]:
...

### 1d) The Quadratic and the Covariance Matrix ###
You have now checked in a couple of ways that if the two-dimensional random vector $\mathbf{Z}$ consists of i.i.d. standard normal variables, then the joint density of $\mathbf{Z}$ is

$$
f(\mathbf{z}) 
~ = ~ \frac{1}{(\sqrt{2\pi})^2} \exp(-\frac{1}{2}\mathbf{z}^T \mathbf{z})
$$

The key point to keep in mind:

**When $\mathbf{Z}$ is i.i.d. standard normal, the quadratic in the exponent of the density function is $\frac{1}{2}\mathbf{z}^T\mathbf{z}$. That is, the quadratic is half the dot product of $\mathbf{z}$ with itself.**

Because $Z_1$ and $Z_2$ are i.i.d. standard normal, the covariance matrix of $\mathbf{Z}$ is the two-dimensional identity matrix $\mathbf{I}_2$, which `SymPy` has chosen to name as follows:

In [ ]:
eye(2)

After you've stopped laughing, check that `SymPy` gives the right answers for the following. We are using the notation $\det(\mathbf{M})$ for the determinant of $\mathbf{M}$.

(i) $\mathbf{I}_2^{-1}$

(ii) $\det(\mathbf{I}_2)$

(iii) $\mathbf{z}^T \mathbf{I}_2^{-1} \mathbf{z}$

For a `SymPy` matrix `mat`:
- `mat.T` gives the transpose
- `mat.det()` gives the determinant
- `mat.inv()` gives the inverse

In [ ]:

# (i)

...

In [ ]:

# (ii)

...

In [ ]:

# (iii)

...

You showed earlier that we can rewrite the i.i.d. standard normal joint density as

$$
f(\mathbf{z}) 
~ = ~ \frac{1}{(\sqrt{2\pi})^2} \exp(-\frac{1}{2}\mathbf{z}^T \mathbf{z})
$$

Now you can write another equivalent form:

$$
f(\mathbf{z}) 
~ = ~ \frac{1}{(\sqrt{2\pi})^2\sqrt{\det(\mathbf{I}_2)}} \exp(-\frac{1}{2}\mathbf{z}^T \mathbf{I}_2^{-1} \mathbf{z})
$$

We say that $\mathbf{Z}$ is has the **centered bivariate normal distribution with covariance matrix $\mathbf{I}_2$**. 

To see why we complicating matters by throwing in a bunch of silly multiplications by 1, continue with the lab.

#newpage

## Part 2: Changing Units – The Effect on the Quadratic  ###
Let $a$ and $b$ be non-zero constants. Let $V_1 = aZ_1$ and $V_2 = bZ_2$, and let

$$
\mathbf{V} ~ = ~ 
\begin{bmatrix}
V_1 \\
V_2
\end{bmatrix}
$$

Let $f_\mathbf{V}$ be the joint density of $V_1$ and $V_2$. We will say for short that $f_\mathbf{V}$ is the joint density of $\mathbf{V}$.

The goal of this part of the lab is to begin to write the formula for $f_\mathbf{V}$ in terms of matrices and to understand the quadratic component.

Much of the work in this part can be done easily by hand. But setting up the matrix framework in `SymPy`, as you are being asked to do below, will make some of the subsequent parts easier.

### 2a) Joint Density of a Linear Transformation ###

We started the lab with the definition 

$$
f(z_1, z_2) ~ = ~ \frac{1}{2\pi} e^{-\frac{1}{2}(z_1^2 + z_2^2)}
$$

Fill in the three blanks below by extending the argument for linear change of variable in one dimension. 

$$
f_{\mathbf{V}}(v_1, v_2) ~ = ~ 
f \big{(} \underline{ ~ (i) ~ }, \underline{ ~ (ii) ~ } \big{)} \cdot \frac{1}{\underline{ ~ (iii) ~ }}
$$

Justify your expression **on paper** by writing a formula for $f_{\mathbf{V}}(v_1, v_2)$ in terms of $f$, $a$, $b$ and of course $v_1$ and $v_2$. Be careful – make sure that your density can't be negative.


(i)

(ii)

(iii)

### 2b) Image and Preimage ###
The random vector $\mathbf{V}$ is a linear transformation of $\mathbf{Z}$, so $\mathbf{V} = \mathbf{A}\mathbf{Z}$ where $\mathbf{A}$ is a matrix of coefficients.

On the space of possible values, let vectors $\mathbf{z}$ and $\mathbf{v}$ be such that

$$
\mathbf{v} ~ = ~ \mathbf{A}\mathbf{z}
$$

We will call $\mathbf{v}$ the *image* of $\mathbf{z}$ under the linear tranformation by $\mathbf{A}$, and we will call $\mathbf{z}$ the *preimage* of $\mathbf{v}$.

Informally, $\mathbf{z}$ is the point that leads to $\mathbf{v}$ via the linear transformation $\mathbf{A}$.

In this exercise you will establish the relation between the preimage and the quadratic in the exponent. First, run the cell below to `declare` some variables.

In [ ]:
declare('a')
declare('b')
declare('v_1')
declare('v_2')

You can use `Matrix` to create a matrix by rows, providing a list of lists as the argument, as follows. 

Here is a matrix with just one row. Notice the contrast with the way of producing a column.

In [ ]:
Matrix([[1,2]]), Matrix([1, 2])

In [ ]:
Matrix([[1, 2], [3, 4]])

Create the matrix $\mathbf{A}$ in `SymPy`. Call it `coeffs` as a reminder that it contains the coefficients of the linear transformation.

In [ ]:

coeffs = ...
coeffs

Remember that you have already defined
$
\mathbf{z} ~ = ~ 
\begin{bmatrix}
z_1 \\
z_2
\end{bmatrix}
$

Find the image of $\mathbf{z}$. This is a check to see that your definition of `coeffs` is correct.

In [ ]:

image_of_z = ...
image_of_z

Define
$
\mathbf{v} ~ = ~ 
\begin{bmatrix}
v_1 \\
v_2
\end{bmatrix}
$

In [ ]:

v = ...
v

Find the preimage of $\mathbf{v}$. Make sure your answer is a column vector. You might want to use some scratch paper first.

In [ ]:

preimage_of_v = ...
preimage_of_v

Find the dot product of this preimage with itself.

In [ ]:

...

Go back and look at your answer to **2a**. 

**True or false:**

In the formula for the joint density $f_\mathbf{V}$ at the point $\mathbf{v}$, the quadratic in the exponent is negative half the dot product of the preimage of $\mathbf{v}$ with itself.


**Your answer here:**


### 2c) Covariance Matrix of $\mathbf{V}$ ###

Let $\boldsymbol{\Sigma}_\mathbf{V}$ be the covariance matrix of $\mathbf{V}$. Define $\boldsymbol{\Sigma}_\mathbf{V}$ in terms of $a$ and $b$.

In [ ]:

Sigma_V = Matrix([... , ...])
Sigma_V

Find $\boldsymbol{\Sigma}_\mathbf{V}^{-1}$.

In [ ]:

...

Find $\mathbf{v}^T \boldsymbol{\Sigma}^{-1}_\mathbf{V} \mathbf{v}$.

In [ ]:

...

So in the formula for the joint density of $f_\mathbf{V}$ at the point $\mathbf{v}$, the quadratic in the exponent can be written as

$$
-\frac{1}{2} \mathbf{v}^T \boldsymbol{\Sigma}_\mathbf{V}^{-1} \mathbf{v}
$$

This form is useful because it is expressed directly in terms of $\mathbf{v}$ and not the preimage. 

#newpage

## Part 3. Changing Units: The Effect on the Constant of Integration ##
To complete the formula for $f_\mathbf{V}$, we need the constant of integration.

Recall from your linear algebra class that in two dimensions, the determinant of a matrix is the area of the parallelogram formed by the images of the two unit vectors under the transformation by that matrix.

Let's confirm this for the transformation in Part 2.

### 3a) Transformation of the Unit Square ###
Define the two unit vectors. Remember that they should be column vectors.

In [ ]:

uv_1 = ...
uv_2 = ...
uv_1, uv_2

Find the images of the unit vectors under the tranformation by `coeffs`.

In [ ]:

image_1 = ...
image_2 = ...
image_1, image_2

For $a=2$ and $b=3$, here is the unit square and the resulting parallelogram (which is in this case a rectangle). The area of the unit square has gone up by a factor of $ab = 6$.

In [ ]:
unit_square_to_rectangle()

### 3b) Two Related Determinants ###
For the transformation $\mathbf{V} = \mathbf{AZ}$ of Part 2, 

$$
\boldsymbol{\Sigma}_\mathbf{V} ~ = ~ \mathbf{AA}^T
$$

and if $\mathbf{z}$ is the preimage of $\mathbf{v}$ then

$$
\mathbf{z}^T\mathbf{z} ~ = ~ \mathbf{v}^T (\mathbf{A^TA})^{-1} \mathbf{v}
~ = ~ \mathbf{v}^T \boldsymbol{\Sigma}_\mathbf{V}^{-1} \mathbf{v}
$$

You can check these facts algebraically on scratch paper using the `coeffs` and `Sigma_V` matrices that you defined in Part 2. Or you can wait till we prove more general versions in class.

Use properties of determinants and the first fact above to explain why $\det(\boldsymbol{\Sigma}_\mathbf{V}) = (\det(\mathbf{A}))^2$.


**Your answer here:**

Find $\det(\boldsymbol{\Sigma}_\mathbf{V})$ in the cell below.

In [ ]:

...

Now look at what you put in Blank (iii) of **2a**, and fill in the blanks below.

I filled Blank (iii) of **2a** with the $\underline{~~~~~~~~~~~~~~~~}$ 
of the determinant of $\mathbf{A}$, which is the same as the 
$\underline{~~~~~~~~~~~~~~~~}$ 
of the determinant of $\boldsymbol{\Sigma_\mathbf{V}}$.


**Your answer here:**



### 3c) A Reparametrization ###

Your work in Parts 2 and 3 has shown that

$$
f_\mathbf{V}(\mathbf{v}) ~ = ~ \frac{1}{(\sqrt{2\pi})^2 \sqrt{\det(\boldsymbol{\Sigma}_\mathbf{V})} } \exp \big{(} -\frac{1}{2} \mathbf{v}^T \boldsymbol{\Sigma}_\mathbf{V}^{-1} \mathbf{v} \big{)}
$$

The joint density has been expressed in terms of the parameter $\boldsymbol{\Sigma}$, the covariance matrix. We say that $\mathbf{V}$ has the **centered bivariate normal distribution with covariance matrix $\boldsymbol{\Sigma}_\mathbf{V}$**.

Confirm that the formula at the end of Part 1 is a special case of this one.

This way of expressing the joint density in terms of the covariance matrix extends to higher dimensions. The formula is exactly the same.

### 3d) Visualization ###
Run the cell below to see graphs of $f$ and $f_\mathbf{V}$. As in **3a**, we have set $a=2$ and $b=3$. The bottom left panel shows 10 points picked independently according to $f$. The bottom right panel shows those 10 points transformed to create the corresponding 10 replications from $f_\mathbf{V}$.

As you know, $f$ has circular symmetry – its level sets are circles. Those circles get transformed into ellipses whose axes are along the $v_1$ and $v_2$ axes. That is why the level sets $f_\mathbf{V}$ are ellipses. Move the slider (slowly!) to the right to see the elongated shape.

In [ ]:
z_to_v(2, 3, n=10)

#newpage

## Part 4: Introducing Dependence ##

Let $Z_1$ and $Z_2$ be i.i.d. standard normal as before. Now Let $W_1 = Z_1$ and $W_2 = aZ_1 + bZ_2$. As before, both $a$ and $b$ are non-zero constants. 

Let 
$\mathbf{W} = 
\begin{bmatrix}
W_1 \\
W_2
\end{bmatrix}
$. Our goal is to find the joint density $f_\mathbf{W}$ at the generic point $\mathbf{w} = 
\begin{bmatrix}
w_1 \\
w_2
\end{bmatrix}
$.

### 4a) Marginals and Covariance ###

Are the marginal distributions of $W_1$ and $W_2$ normal? Why or why not?


**Your answer here:**



Find the covariance matrix $\boldsymbol{\Sigma}_\mathbf{W}$ of the random vector $\mathbf{W}$.

In [ ]:

Sigma_W = Matrix([..., ...])
Sigma_W

Could $W_1$ and $W_2$ be independent? Explain your answer.


**Your answer here:**



### 4b) The Quadratic ###

By the same logic as in **2a**,
$$
f_\mathbf{W}(\mathbf{w}) ~ = ~ Cf(\mathbf{z})
$$
where $\mathbf{z}$ is the preimage of $\mathbf{w}$ and $C$ is a positive constant.

We know that the quadratic in the exponent of $f(\mathbf{z})$ is $\frac{1}{2}\mathbf{z}^T\mathbf{z}$. In this exercise you will express that quadratic in terms of $\mathbf{w}$ and $\boldsymbol{\Sigma}_\mathbf{W}$.

$\mathbf{W} = \mathbf{AZ}$ for some matrix $\mathbf{A}$. Define $\mathbf{A}$ in the cell below; call it `coeffs` as before.

In [ ]:

coeffs = ...
coeffs

Check that your matrix is correct by finding the image of $\mathbf{z}$ under the transformation by the matrix.

In [ ]:

...

Now run the cell below to define
$$
\mathbf{w} ~ = ~ 
\begin{bmatrix}
w_1 \\
w_2
\end{bmatrix}
$$

In [ ]:
declare('w_1')
declare('w_2')
w = Matrix([w_1, w_2])
w

Find the preimage of $\mathbf{w}$.

In [ ]:

preimage = ...
preimage

Find the dot product of the preimage with itself. The quadratic in the exponent of the joint density function is 1/2 times this dot product. 

In [ ]:

preimage_dotprod = ...
preimage_dotprod

As in Part 2, this dot product is equal to $\mathbf{w}^T \boldsymbol{\Sigma}_\mathbf{W}^{-1} \mathbf{w}$. This is called a *quadratic form*. 

Compute the quadratic form, and don't get anxious if it doesn't look like the output of the cell above. Algebraic expressions can be written in many equivalent ways.

In [ ]:

quad_form = ...
quad_form

The function `rearrange_1` saves you the trouble of simplifying the algebra. Run the two cells below and compare the outputs.

In [ ]:
rearrange_1(preimage_dotprod)

In [ ]:
rearrange_1(quad_form)

What you have shown thus far is that
$$
f_\mathbf{W}(\mathbf{w}) ~ = ~ C \exp (-\frac{1}{2} \mathbf{w}^T \boldsymbol{\Sigma}_\mathbf{W}^{-1} \mathbf{w})
$$

### 4c) The Constant of Integration ###
As in Part 3, the constant $C$ has two factors. One factor is $(1/\sqrt{2\pi})^2$. The other is $1/s$ where $s$ is the area of the parallelogram formed by the transformed unit vectors.

Remember that the transformation is determined by the matrix `coeffs`, and that the unit vectors are called `uv_1` and `uv_2`.

In [ ]:
coeffs, uv_1, uv_2

Find the images of the unit vectors under the linear transformation.

In [ ]:

image_1 = ...
image_2 = ...
image_1, image_2

The figure below shows the two image vectors and the resulting parallelogram in the case $a=3$ and $b=1.5$.

In [ ]:
unit_square_to_parallelogram()

Fill in the blank (it's fine to assume $a$ and $b$ are positive):

If we define the "base" of the parallelogram to be a vertical side of length b, then the "height" of the red parallelogram is always equal to $\underline{~~~~~~~~}$.


**Your answer here:**



Find $\det(\boldsymbol{\Sigma}_\mathbf{W})$.

In [ ]:

...

**True or False:** The area of the parallelogram is $\sqrt{\det(\boldsymbol{\Sigma}_\mathbf{W})}$.


**Your answer here:**


### 4d) The Joint Density ###
You have shown that $\mathbf{W}$ has the **centered bivariate normal density with covariance matrix $\boldsymbol{\Sigma}_\mathbf{W}$** given by

$$
f_\mathbf{W}(\mathbf{w}) ~ = ~ \frac{1}{(\sqrt{2\pi})^2 \sqrt{\det(\boldsymbol{\Sigma}_\mathbf{W})} } \exp \big{(} -\frac{1}{2} \mathbf{w}^T \boldsymbol{\Sigma}_\mathbf{W}^{-1} \mathbf{w} \big{)}
$$

Run the cell below for graphs of $f$ and $f_\mathbf{W}$. As in **4c**, we have set $a=3$ and $b=1.5$. Circles get transformed to ellipses as before, but now the axes of the ellipses are at an angle to the $w_1$ and $w_2$ axes.

In [ ]:
z_to_w(3, 1.5, n=10)

At this point you should be pretty confident that if we took some other centered linear transformation of $\mathbf{Z}$, that too would have a joint density of the form that we have developed. We will do the general calculation in class. Because you have done this lab, you will understand what the pieces of the formula mean.

## Endnote: $n$ Dimensions ##
We will discuss the details in class, but here is a summary that should come as no surprise.

Let $\mathbf{X} = 
\begin{bmatrix}
X_1 \\
X_2 \\
\vdots \\
X_n
\end{bmatrix}
$ be a random vector.

- For a symmetric positive definite $n \times n$ matrix $\boldsymbol{\Sigma}$, the random vector $\mathbf{X}$ has the centered multivariate normal distribution with covariance matrix $\boldsymbol{\Sigma}$ if the joint density function is given by

$$
f_{\mathbf{X}}(\mathbf{x}) ~ = ~ \frac{1}{(\sqrt{2\pi})^n \sqrt{\det(\boldsymbol{\Sigma})} } \exp \big{(} -\frac{1}{2} \mathbf{x}^T \boldsymbol{\Sigma}^{-1} \mathbf{x} \big{)}
$$

- Multivariate normal variables with mean vector $\boldsymbol{\mu}$ and covariance matrix $\boldsymbol{\Sigma}$ are just $Y_1, Y_2, \ldots, Y_n$ where $Y_i = X_i + \boldsymbol{\mu}[i]$. Shifting doesn't change covariances or volumes. So

$$
f_{\mathbf{Y}}(\mathbf{y}) ~ = ~ f_{\mathbf{X}}(\mathbf{y} - \boldsymbol{\mu})
~ = ~ \frac{1}{(\sqrt{2\pi})^n \sqrt{\det(\boldsymbol{\Sigma})} } \exp \big{(} -\frac{1}{2} (\mathbf{y} - \boldsymbol{\mu})^T \boldsymbol{\Sigma}^{-1} (\mathbf{y} - \boldsymbol{\mu}) \big{)}
$$

We will see that if $\mathbf{X}$ is multivariate normal then all the marginals are normal, as is the conditional distribution of any of the coordinates given any of the others. However, it is possible to construct $\mathbf{X}$ such that all the marginals are normal but the joint density is not multivariate normal.

## Conclusion ##
You have learned:
- how to use `SymPy` for integration and linear algebra
- the formula for the multivariate normal joint density
- where the pieces of the formula come from
- what the joint density surface looks like in two dimensions
- how the covariance matrix is related to the matrix of coefficients of the linear transformation

In [ ]:
import gsExport
gsExport.generateSubmission("Lab_10.ipynb")